# Compare vaccination strategies. $e$=0.7

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import datetime
from experiments import get_experiments_results
from env_var import EPIDEMIC, MAPPINGS, EXPERIMENTS
from matplotlib.lines import Line2D

mpl.rcParams['text.usetex'] = True
mpl.rcParams['figure.dpi']= 175
plt.style.use('ggplot')

In [2]:
# Simulation parameters
num_age_groups = EXPERIMENTS['num_age_groups']
num_ervas = EXPERIMENTS['num_ervas']
T = EXPERIMENTS['simulate_T']
init_vacc = EXPERIMENTS['init_vacc']
taus = EXPERIMENTS['taus']
u = EXPERIMENTS['vaccines_per_day']
r_experiments = EXPERIMENTS['r_effs']
t0 = EXPERIMENTS['t0']
e = EPIDEMIC['e']
strategies = EXPERIMENTS['strategies']
u_offset = EXPERIMENTS['u_offset']

# Plot parameters
plot_interval = 10
ervas_labels = EPIDEMIC['ervas_order']
age_labels = MAPPINGS['age_groups'][num_age_groups]['names']
color_palette = ["#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
len_bar = 5
sep_bet_group = 4*len_bar
legend_width = 4

begin = datetime.datetime.strptime(t0, '%Y-%m-%d')
ts = [begin + datetime.timedelta(days=day) for day in range(T)]
print('Simulation ends: %s' % (ts[-1]))
t2 = int(T-1-u_offset)
t1 = t2-30

Simulation ends: 2021-08-10 00:00:00


In [ ]:
exp_results = get_experiments_results(num_age_groups=num_age_groups,
                                      num_ervas=num_ervas,
                                      init_vacc=init_vacc,
                                      strategies=strategies,
                                      u=u,
                                      T=T,
                                      r_experiments=r_experiments,
                                      t0=t0,
                                      e=e,
                                      taus=taus,
                                      u_offset=u_offset)

Beginning experiments.
Parameters:
Number of age ervas: 5.
Number of age groups: 9.
Number of vaccines per day: 30000.
t0: 2021-04-18.
T: 115.
Vaccine efficacy (e): 0.7.
Rs to try: [0.75, 1.0, 1.25, 1.5].
Taus to try: [0, 0.5, 1.0].
Initialize with vaccinated people: True.
Strategies:
[([1, 0, 0], 'Pop only'), ([0, 0, 0], 'No vaccination'), ([0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'Pop, Inf and Hosp'), ([0.5, 0, 0.5], 'Pop and Hosp'), ([0.5, 0.5, 0], 'Pop and Inf'), ([0, 0.5, 0.5], 'Hosp and Inf'), ([0, 0, 1], 'Hosp only'), ([0, 1, 0], 'Inf only'), (True, 'Optimized Death'), (False, 'Optimized Hosp')].

Running 112 experiments with 4 CPUS.
Start (56523). Exp: 1. R: 0.75. tau: 0. Policy: Pop onlyStart (56524). Exp: 2. R: 0.75. tau: 0. Policy: No vaccinationStart (56525). Exp: 3. R: 0.75. tau: 0. Policy: Pop, Inf and Hosp
Start (56526). Exp: 4. R: 0.75. tau: 0. Policy: Pop and Hosp


Finished (56523). Exp: 1. Elapsed: 0:00:04.124613
Finished (56525). Exp: 3. Elapsed

In [ ]:
def plot_results(ax, r, tau, metric, exp_results, T, erva, ervas_labels, u_offset,
                 plot_interval, baseline='Pop only', skip_no_vacc=True, out_sort=True):
    # Check the index of the ERVA we are plotting
    try:
        erva_idx = ervas_labels.index(erva)
    except ValueError:
        if erva == 'Finland':
            erva_idx = -1
        else:
            raise ValueError('Wrong ERVA selected')
    
    # Get only the elements starting from the offset
    x = np.arange(T)
    x = x[u_offset:]

    # Look for the baseline result for the current r, tau
    results = exp_results[r][tau]
    baseline_all_time = None
    for label, label_level in results.items():
        if label == baseline:
            baseline_results = label_level['results'][metric]
            # Sum across all age groups
            baseline_all_time = baseline_results.sum(axis=0)
            if erva_idx == -1:
                # Sum across all ERVAs
                baseline_all_time = baseline_all_time.sum(axis=0)
            else:
                baseline_all_time = baseline_all_time[erva_idx, :]
            baseline_all_time = baseline_all_time[u_offset:]
    
    assert baseline_all_time is not None
    assert baseline_all_time.shape[0] == len(x)
    
    # Iterate over all the strategies
    totals = []
    label_i = 0
    for label, label_level in results.items():
        if label == baseline:
            continue

        if label == 'No vaccination' and skip_no_vacc:
            continue
            
        metric_results = label_level['results'][metric]
        # Sum across all age groups
        metric_all_time = metric_results.sum(axis=0)
        if erva_idx == -1:
            # Sum across all ERVAs
            metric_all_time = metric_all_time.sum(axis=0)
        else:
            metric_all_time = metric_all_time[erva_idx, :]
        
        if 'Optimized' not in label:
            metric_all_time = metric_all_time[u_offset:]
        
        assert metric_all_time.shape[0] == len(x)
        
        # Get the relative change wrt to the baseline
        plot_metric = metric_all_time/baseline_all_time
        plot_metric = np.nan_to_num(plot_metric, nan=1)
        plot_metric = plot_metric - 1
        plot_metric = plot_metric*100
        
        # Plot the temporal line
        color = label_i % len(color_palette)
        ax.plot(x, plot_metric, label=label, color=color_palette[color])
        
        # Get also the result over all time so then produce sorted results
        baseline_policy = baseline_all_time.sum(axis=0)
        total_policy = metric_all_time.sum(axis=0)
        rel_total = ((total_policy/baseline_policy)-1)*100
        total = (label, rel_total)
        totals.append(total)
        label_i += 1
    
    # Print the sorted results
    if out_sort:
        sorted_totals = sorted(totals, key=lambda x: x[-1])
        for total in sorted_totals:
            print('%s: %s' % (total[0], total[1]))
    
    # Plot the labels
    ax.set_xlabel('Days after start time')
    ax.set_ylabel('Relative change in {0} (\%)'.format(metric))
    ax.set_title('Relative change of %s in %s compared to %s.\n R=%s. tau=%s' % (metric, erva, baseline, r, tau))
    ax.legend()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(plot_interval))
        
    return ax

# Deaths

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=1.5, tau=0.5, metric='deaths', exp_results=exp_results, T=T,
                  erva='Finland', ervas_labels=ervas_labels, plot_interval=plot_interval,
                  u_offset=u_offset)

# Hospitalizations

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=1.5, tau=0.5, metric='new hospitalizations', exp_results=exp_results, T=T,
                  u_offset=u_offset, erva='Finland', ervas_labels=ervas_labels, plot_interval=plot_interval)

# Infections

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=1.5, tau=0.5, metric='infections', exp_results=exp_results, T=T,
                  u_offset=u_offset, erva='Finland', ervas_labels=ervas_labels, plot_interval=plot_interval)

# Saving figures and generating latex

In [ ]:
def save_and_generate_timeplot(r_experiments, taus, ervas_labels, metrics, exp_results, skip_no_vacc, out_sort, dpis,
                            u_offset, T, img_dir, plot_interval, baseline, file_format, latex_template):
    os.makedirs(img_dir, exist_ok=True)
    plot_ervas = ['Finland']+ ervas_labels
    # Iterate over all the results
    for r in r_experiments:
        for tau in taus:
            for metric in metrics:
                # Plot in a single figure the values for Finland and all the ERVAs
                fig, axs = plt.subplots(2, 3,
                                        sharex=True,
                                        figsize=(16, 11))
                axs = axs.flat
                # Iterate across all regions
                for erva, ax in zip(plot_ervas, axs):
                    ax = plot_results(ax, r=r, tau=tau, metric=metric, exp_results=exp_results, T=T,
                                      erva=erva, ervas_labels=ervas_labels,
                                      skip_no_vacc=skip_no_vacc, u_offset=u_offset, plot_interval=plot_interval,
                                      baseline=baseline, out_sort=out_sort)
                    # Remove the legends and axis labels, they are going to be in the figure
                    handles, labels = ax.get_legend_handles_labels()
                    ax.get_legend().remove()
                    x_lab = ax.get_xlabel()
                    ax.set_xlabel('')
                    y_lab = ax.get_ylabel()
                    ax.set_ylabel('')
                    ax.set_title('%s' % (erva, ), y=0.93, fontsize=16)
                # Include in the figure the axis labels and legend
                fig.text(0.5, -0.02, x_lab,
                         ha='center', fontsize=24)
                fig.text(-0.02, 0.5, y_lab,
                         va='center', rotation='vertical', fontsize=24)
                
                ncols = round(len(labels)/2)
                legend = fig.legend(handles, labels, fontsize=18,
                                    loc='upper center', bbox_to_anchor=(0.5, 1.1),
                                    framealpha=0.5, ncol=ncols)
                
                fig.tight_layout()
                
                name = 'r%s_tau%s_metric_%s' % (r, tau, metric)
                # Save the figure
                fig_name = name + '.' + file_format 
                fig_path = os.path.join(img_dir, fig_name)
                fig.savefig(fig_path, format=file_format, dpi=dpis, bbox_inches='tight')
                # Construct the latex code for this figure
                latext_text = latex_template.format(fig_name=fig_name,
                                                    metric=metric,
                                                    baseline=baseline,
                                                    name=name,
                                                    r=r,
                                                    tau=tau)
                print(latext_text)
                
                

In [ ]:
latex_template = """
    \\begin{{figure}}[p]
      \\centering
      \\includegraphics[width=\\textwidth]{{img/{fig_name}}}
      \\caption{{Relative change in {metric} for all vaccination strategies with respect to vaccination strategy {baseline}.
      For this scenario, the basic reproduction number $R_0 = {r}$ and the mobility value $\\tau = {tau}$.}}
      \\label{{fig:{name}}}
    \\end{{figure}}
"""
metrics = ['deaths', 'new hospitalizations', 'infections']
save_and_generate_timeplot(r_experiments=r_experiments, taus=[0.5], ervas_labels=ervas_labels, metrics=metrics,
                        exp_results=exp_results, skip_no_vacc=True, u_offset=u_offset, file_format='pdf',
                        T=T, img_dir=os.path.join('out', 'img'), latex_template=latex_template,
                        plot_interval=plot_interval, baseline='Pop only', out_sort=False, dpis=2000)

# Print all $\beta$ and Rs

In [ ]:
taus = EXPERIMENTS['taus']
for r in r_experiments:
    for tau in taus:
        r_tau_res = exp_results[r][tau]
        any_key = list(r_tau_res.keys())[0]
        beta = exp_results[r][tau][any_key]['parameters']['beta']
        print('R: %s. Tau: %s. Beta: %s' % (r, tau, beta))

# Compare vaccination per age groups and ERVAs

In [ ]:
def plot_bars_ages_ervas(ax, r, tau, metric, exp_results, labels, plot, u_offset,
                         t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True):
    # Get the results only for the specified r and tau
    results = exp_results[r][tau]
    
    # Count how many strategies we have
    total_strategies = len(results.keys())
    skip_num = 1
    if skip_no_vacc:
        skip_num += 1
    # Space for a set of bars (for one ERVA or Age group)
    space_label = len_bar*(total_strategies-skip_num) + sep_bet_group
    
    # Find the strategy acting as baseline and get the results for it
    baseline_cumm = None
    for label, label_level in results.items():
        if label == baseline:
            baseline_results = label_level['results'][metric]
            baseline_cumm = baseline_results.cumsum(axis=2)

            if plot == 'ERVA':
                baseline_cumm = baseline_cumm.sum(axis=0)
            elif plot == 'Age group':
                baseline_cumm = baseline_cumm.sum(axis=1)
            else:
                raise ValueError('Not valid value to plot')
                
            baseline_cumm = baseline_cumm[:, u_offset:]
    
    assert baseline_cumm is not None
    
    # Reserve space for the complete plot (all ERVAs or Age groups)
    total_plot_space = space_label*len(labels) + sep_bet_group
    ax.set_xlim(0, total_plot_space)
    
    # Iterate over the strategies, get result wrt baseline and plot
    num_strategy = 0
    legend_elements = []
    for label, label_level in results.items():
        if label == baseline:
            continue
        
        if label == 'No vaccination' and skip_no_vacc:
            continue

        # Get the results for this strategy
        policy_results = label_level['results'][metric]
        policy_cumm = policy_results.cumsum(axis=2)

        if plot == 'ERVA':
            policy_cumm = policy_cumm.sum(axis=0)
        elif plot == 'Age group':
            policy_cumm = policy_cumm.sum(axis=1)
        else:
            raise ValueError('Not valid value to plot')
        
        if 'Optimized' not in label:
            policy_cumm = policy_cumm[:, u_offset:]

                
        assert policy_cumm.shape == baseline_cumm.shape
        assert policy_cumm.shape[0] == len(labels)

        # Obtain the the results for all the ERVAs and age groups
        relative_to_baseline = policy_cumm/baseline_cumm
        relative_to_baseline = np.nan_to_num(relative_to_baseline, nan=1)
        relative_to_baseline = relative_to_baseline - 1
        relative_to_baseline = relative_to_baseline*100

        # Get the results between t1 and t2
        min_reduction = relative_to_baseline[:, t1]
        max_reduction = relative_to_baseline[:, t2]


        color_i = num_strategy % len(color_palette)
        color = color_palette[color_i]
        # Iterating the ERVAs or Age groups.
        for label_i in range(len(labels)):
            # Getting the the dimensions of the bar
            x1_square = label_i*space_label + sep_bet_group + num_strategy*len_bar
            y1_square = min_reduction[label_i]
            y2_square = max_reduction[label_i] - min_reduction[label_i]

            # Plot the result of this policy to the current ERVA or Age group
            ax.broken_barh([(x1_square, len_bar), ], (y1_square, y2_square),
                           color=color)

        # Add one element to the legends for this strategy
        legend_elements.append(Line2D([0], [0], color=color, lw=legend_width, label=label))
        num_strategy += 1
    
    # Getting the positions for the ticks and labeling the labels for them
    space_bars = space_label - sep_bet_group
    middle_of_space = space_bars/2
    pos_tick =  sep_bet_group + middle_of_space
    ticks = []
    for label_i in range(len(labels)):
        abs_pos_tick = label_i*space_label + pos_tick
        ticks.append(abs_pos_tick)
        
    assert len(ticks) == len(labels)
    assert len(legend_elements) == total_strategies-skip_num
    
    # Add ticks and labels
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)
    ax.set_xlabel(plot)
    ax.set_ylabel('Reduction in percentage')
    ax.set_title('Reduction in %s at %ss per policy . R=%s' % (metric, plot, r))
    ax.legend(handles=legend_elements, loc='upper left', framealpha=0.5)
    

    return ax

# Deaths

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='deaths', exp_results=exp_results,
                          labels=ervas_labels, u_offset=u_offset,
                          plot='ERVA', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='deaths', exp_results=exp_results,
                          labels=age_labels, u_offset=u_offset,
                          plot='Age group', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

# Hospitalizations

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='new hospitalizations', exp_results=exp_results,
                          labels=ervas_labels, u_offset=u_offset,
                          plot='ERVA', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='new hospitalizations', exp_results=exp_results,
                          labels=age_labels, u_offset=u_offset,
                          plot='Age group', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

# Infections

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='infections', exp_results=exp_results,
                          labels=ervas_labels, u_offset=u_offset,
                          plot='ERVA', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_ervas(ax, r=1.5, tau=0.5, metric='infections', exp_results=exp_results,
                          labels=age_labels, u_offset=u_offset,
                          plot='Age group', t2=t2, t1=t1, baseline='Pop only', skip_no_vacc=True)

# Saving figures and generating latex

In [ ]:
def save_and_generate_bars(r_experiments, taus, ervas_labels, age_labels, metrics, exp_results,
                            dpis, u_offset,
                            skip_no_vacc, img_dir, t2, t1, baseline, file_format, latex_template):
    os.makedirs(img_dir, exist_ok=True)
    # Iterate over all results
    for r in r_experiments:
        for tau in taus:
            for metric in metrics:
                fig, axs = plt.subplots(2, 1, figsize=(11, 15))
                axs = axs.flat
                # Plot one figure for ERVA and one for age group per r, tau pair
                for plot, labels, ax in zip(['ERVA', 'Age group'], [ervas_labels, age_labels], axs):
                    ax = plot_bars_ages_ervas(ax=ax, r=r, tau=tau, metric=metric, plot=plot,
                                              u_offset=u_offset,
                                              exp_results=exp_results, labels=labels, t2=t2, t1=t1,
                                              baseline=baseline, skip_no_vacc=skip_no_vacc)
                    ax.set_ylabel('')
                    ax.set_title(None)
                # Setting the labels for the current figure
                y_lab = 'Change in percentage (\%)'
                fig.text(0.04, 0.5, y_lab, va='center', rotation='vertical', fontsize=16)
                # Saving the figure
                name = 'bars_r%s_tau%s_metric_%s' % (r, tau, metric)
                fig_name = name + '.' + file_format 
                fig_path = os.path.join(img_dir, fig_name)
                fig.savefig(fig_path, format=file_format, dpi=dpis, bbox_inches='tight')
                # Constructing Latex code for the current figure and printing it
                latext_text = latex_template.format(fig_name=fig_name,
                                                    metric=metric,
                                                    baseline=baseline,
                                                    name=name,
                                                    r=r,
                                                    tau=tau)
                print(latext_text)

In [ ]:
latex_template = """
    \\begin{{figure}}[p]
      \\centering
      \\includegraphics[width=\\textwidth]{{img/{fig_name}}}
      \\caption{{Relative change in {metric} for all vaccination strategies with respect to vaccination strategy {baseline}.
      The bars represent the relative change in cumulative {metric} between the end of the simulation and 30 days before the end of the simulation.
      For this scenario, the basic reproduction number $R_0 = {r}$ and the mobility value $\\tau = {tau}$.
      Top: Relative change at ERVA level, Bottom: Relative change at age group level.}}
      \\label{{fig:{name}}}
    \\end{{figure}}
"""
metrics = ['deaths', 'new hospitalizations', 'infections']
save_and_generate_bars(r_experiments=r_experiments, taus=[0.5], ervas_labels=ervas_labels,
                        age_labels=age_labels, u_offset=u_offset,
                        metrics=metrics, exp_results=exp_results, skip_no_vacc=True,
                        img_dir=os.path.join('out', 'img'), t2=t2, t1=t1, baseline='Pop only',
                        file_format='pdf', latex_template=latex_template, dpis=2000)

# Generating table with all results

In [ ]:
def generate_results_table(r_experiments, taus, metric, baseline, exp_results, skip_no_vacc):
    table_dictionary = {}
    all_columns = set()
    # Iterate over all experiments
    for r in r_experiments:
        for tau in taus:
            # Group the results by r, tau
            key = (str(r), str(tau))
            table_dictionary[key] = {}
            results = exp_results[r][tau]
            # Iterate over all the strategies and get the counts
            for label, label_level in results.items():
                if label == baseline:
                    baseline_results = label_level['results'][metric]
                    baseline_results = baseline_results.sum()
                    table_dictionary[key]['baseline'] = {}
                    table_dictionary[key]['baseline']['count'] = baseline_results
                elif label == 'No vaccination' and skip_no_vacc:
                    continue
                else:
                    # Add to columns since this is not the baseline
                    all_columns.add(label)
                    strategy_results = label_level['results'][metric]
                    strategy_results = strategy_results.sum()
                    table_dictionary[key][label] = {}
                    table_dictionary[key][label]['count'] = strategy_results
                    table_dictionary[key][label]['best'] = False
    
    # Iterate over all r, tau combination
    for key, strategies_level in table_dictionary.items():
        best_strategy_val = np.inf
        last_best = None
        # Iterate over all the strategies
        for column in all_columns:
            # Get the difference to the baseline
            strategies_level[column]['diff_to_baseline'] = strategies_level[column]['count'] - strategies_level['baseline']['count']
            # Get the relative change wrt to the baseline
            strategies_level[column]['change_to_baseline'] = strategies_level[column]['count']/strategies_level['baseline']['count'] - 1
            strategies_level[column]['change_to_baseline'] = strategies_level[column]['change_to_baseline']*100
            
            # Check if this strategy is the best for the r, tau combination
            if strategies_level[column]['diff_to_baseline'] < best_strategy_val:
                best_strategy_val = strategies_level[column]['diff_to_baseline']
                strategies_level[column]['best'] = True
                if last_best is not None:
                    strategies_level[last_best]['best'] = False
                    
                last_best = column
    
    # Construct the latex tables
    header_latex = """
    \\begin{table}[h]
    \\centering
    \\tiny
    \\begin{tabular}{c|%s}
    \\toprule
    """
    header_latex = header_latex % ('c'*len(all_columns))
    header_titles = '\\bf $R_0, \\tau$ '
    for column in all_columns:
        header_titles += '& \\bf %s ' % column
    complete_header = header_latex + header_titles + '\\\\ \n \\midrule \n'
    
    # Iterate over the results and add them to the latex table
    lines_diff = ''
    lines_change = ''
    for key, strategies_level in table_dictionary.items():
        r_str, tau_str = key
        line_diff = '%s, %s ' % (r_str, tau_str)
        line_change = line_diff
        for column in all_columns:
            if strategies_level[column]['best']:
                line_diff += '& \\bf '
                line_change += '& \\bf '
            else:
                line_diff += '& '
                line_change += '& '
            line_diff += '%.2f ' % (round(strategies_level[column]['diff_to_baseline'], 2))
            line_change += '%.2f ' % (round(strategies_level[column]['change_to_baseline'], 2))
        line_diff += ' \\\\ \n'
        line_change += ' \\\\ \n'
        
        lines_diff += line_diff
        lines_change += line_change
    
    # Finish the tables with the captions
    tbl_diff = complete_header + lines_diff
    tbl_change = complete_header + lines_change
    
    end_table = """
    \\bottomrule
    \\end{tabular}
    """
    
    tbl_diff += end_table
    tbl_change += end_table
    
    tbl_diff += """
    \\caption{\\label{tab:ResultsRTauDiff} Difference in %s of each vaccination strategy compared to strategy Pop only.
    Each row $p$ contains the results for a pair of parameters $R_0$ and $\\tau$ while each column $s$ for a vaccination strategy.
    The value in cell $p,s$ is the difference in %s between strategy $s$ and Pop only at scenario with parameters $p$.
    The lowest difference at row $p$ is indicated in bold face.}
    \\end{table}
    """
    tbl_diff = tbl_diff % (metric, metric)
    
    tbl_change += """
    \\caption{\\label{tab:ResultsRTauDiff} Relative change in %s with each vaccination strategy compared to strategy Pop only.
    Each row $p$ contains the results for a pair of parameters $R_0$ and $\\tau$ while each column $s$ for a vaccination strategy.
    The value in cell $p,s$ is the relative change in %s with strategy $s$ wrt. Pop only at scenario with parameters $p$.
    The lowest relative change at row $p$ is indicated in bold face.}
    \\end{table}
    """
    tbl_change = tbl_change % (metric, metric)

    return tbl_diff, tbl_change

In [ ]:
metrics = ['deaths', 'new hospitalizations', 'infections']
for metric in metrics:
    tbl_diff, tbl_change = generate_results_table(r_experiments, taus, metric=metric, baseline='Pop only',
                                                  exp_results=exp_results, skip_no_vacc=True)
    print(tbl_diff)
    print(tbl_change)